# NLU Homework 4
## Alex Pine (akp258@nyu.edu)
## 2016/11/25

### Problem 1: Data Preparation
I modified Tensorflow's [data_utils.py](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/models/rnn/translate/data_utils.py) code to prepare the Japanese and English sentence pairs for use with the translation code. The code required very few changes to be used with this data set. The only large changes required were to delete the code used for downloading the original dataset, point it to my local data files instead, and modify the language pair from (English, French) to (Japanese, English).